In [5]:
'''
Extracts emme data into csv's for further processing and analysis
'''

import os
import json
import pandas as pd
import numpy as np

from time import time

import inro.modeller as _m
import inro.emme.datatable as _dt
import inro.emme.database as _db
import inro.emme.desktop.worksheet as _ws
import inro.emme.desktop.app as _ap

start_t = time()

mm = _m.Modeller()
desktop = mm.desktop
de = desktop.data_explorer()

imp_nwp = mm.tool('tmg.input_output.import_network_package')
imp_mat = mm.tool('tmg.input_output.import_binary_matrix')
exp_mat = mm.tool('inro.emme.data.matrix.export_matrices')
ext_att = mm.tool('inro.emme.data.extra_attribute.create_extra_attribute')
shortest_path = mm.tool('inro.emme.network_calculation.shortest_path')

In [6]:
# USER INPUTS
# run specs
# update/check this cell before running

run_properties = {
    
    1:{
        'scen_start': 1,
        'mat_start': 1,
        'run_name': 'Base_2016',
        'run_date': '20210707',
        'scen_folder': r'2016 Base Lakeshore TPAP Final - July 7, 2021'
    },

    2:{
        'scen_start': 11,
        'mat_start': 51,
        'run_name': 'Scen1_2041',
        'run_date': '20210712',
        'scen_folder': r'2041 Scen 1 Lakeshore TPAP Final - July 12th, 2021'
    },
    
    3:{
        'scen_start': 21,
        'mat_start': 101,
        'run_name': 'Scen2a_2041',
        'run_date': '20210712',
        'scen_folder': r'2041 Scen 2a Lakeshore TPAP Final - July 12th, 2021'
    },
    
    4:{
        'scen_start': 31,
        'mat_start': 151,
        'run_name': 'Scen2b_2041',
        'run_date': '20210712',
        'scen_folder': r'2041 Scen 2b Lakeshore TPAP Final - July 12th, 2021'
    },
    5:{
        'scen_start': 41,
        'mat_start': 201,
        'run_name': 'Scen0_2041',
        'run_date': '20210803',
        'scen_folder': r'2041 Scen 0 Lakeshore TPAP Final - Aug 3rd, 2021'
    },
    6:{
        'scen_start': 51,
        'mat_start': 251,
        'run_name': 'Scen3_2041',
        'run_date': '20210909',
        'scen_folder': r'2041 Scen 3 Lakeshore TPAP Final - Sept 9, 2021'
    }
    
}

# True or False, run or don't run
process_nwps = False
process_matrices = False
process_auto_metrics = True
process_transit_metrics = False


In [ ]:
# constant specs (they don't change from run to run)

# directories

data_dir = r'C:\Users\ANLIU\OneDrive - HDR, Inc\Documents\5.0_Project_Dev\5.2.3 Traffic - Transit\Macro Model Processing\0 Model Outputs'
out_dir = r'C:\Users\ANLIU\OneDrive - HDR, Inc\Documents\5.0_Project_Dev\5.2.3 Traffic - Transit\Macro Model Processing\1 Processed Outputs'

emmebank_dir = os.path.dirname(_m.Modeller().emmebank.path)

# period specs
with open(os.path.join(out_dir, 'period_specs.json')) as period_specs_file:
    periods = json.load(period_specs_file)

period_order = [
    'AM',
    'MD',
    'PM',
    'EV',
    'ON'
]

# nodes along lakeshore by scenario
with open(os.path.join(out_dir, 'lakeshore_nodes.json')) as lakeshore_nodes_file:
    lakeshore_nodes_sets = json.load(lakeshore_nodes_file)

# nodes and zones in scope

inscope_nodes = pd.read_csv(os.path.join(out_dir,'inscope_nodes.csv'))
inscope_nodes = inscope_nodes.node.tolist()
inscope_zones = pd.read_csv(os.path.join(out_dir,'inscope_zones.csv'))
inscope_zones = inscope_zones.zone.tolist()

# screenlines
screenlines = pd.read_csv(os.path.join(out_dir,'screenlines.csv'))

In [7]:
# the actual calculations and stuff

def process_outputs(run_properties):
    
    # run properties
    scen_start = run_properties['scen_start']
    mat_start = run_properties['mat_start']
    run_name = run_properties['run_name']
    run_date = run_properties['run_date']
    scen_folder = run_properties['scen_folder']
    
    lakeshore_nodes = lakeshore_nodes_sets[run_name]
    
    # output folder for this run
    processed_dir = os.path.join(out_dir,run_name + ' ' + run_date)
    if not os.path.isdir(processed_dir):
        os.mkdir(processed_dir) 
    
    # scenrio-specific input directories
    nwp_dir = os.path.join(data_dir, scen_folder, 'FinalNetworks')
    mat_dir_LOS = os.path.join(data_dir, scen_folder, 'LOS Matrices')        
    mat_dir_dem = os.path.join(data_dir, scen_folder, 'Demand')

    # import network packages

    if process_nwps:

        nwp_files = [
            'AMRoad.nwp',
            'AMTransit.nwp',
            'MDRoad.nwp',
            'MDTransit.nwp',
            'PMRoad.nwp',
            'PMTransit.nwp',
            'EVRoad.nwp',
            'EVTransit.nwp',
            'ONRoad.nwp',
        ]

        scen = scen_start

        for f in nwp_files:
            'Processing nwp for {}'.format(f)
            
            nwp_file = os.path.join(nwp_dir,f)
            imp_nwp.NetworkPackageFile = nwp_file
            imp_nwp.ScenarioId = scen
            imp_nwp.OverwriteScenarioFlag = True
            imp_nwp.ScenarioName = run_name + f[:len(f)-4]
            imp_nwp.ScenarioDescription = run_name + '_' + run_date + '_' + f[:len(f)-4]

            imp_nwp.run()

            scen += 1

        app = _ap.connect()
        app.refresh_data()
        
    # import matrices and save to file

    if process_matrices:
        
        mat_no = mat_start # count start (each run has 36)

        mat_files_LOS = [
            'acost',
            'aivtt',
            'tivtt',
            'twait',
            'twalk'
        ]
        
        mat_files_dem = [
            'AutoMatrix',
            'BicycleMatrix',
            'TransitMatrix',
            'WalkMatrix',
            'LightMatrix',
            'MediumMatrix',
            'HeavyMatrix'
        ]

        scen = scen_start

        for p in period_order:

            scenario_id = scen
            emmebank = inro.modeller.Modeller().emmebank
            scenario = emmebank.scenario(scenario_id)

            # set primary scenario
            de.replace_primary_scenario(scenario)
            
            # LOS matrices
            for m in mat_files_LOS:
                
                'Processing LOS matrices for {}, {}'.format(p,m)

                if not (p == 'ON' and m[0] == 't'): # no transit in ON

                    # import binary matrix
                    mat_file = os.path.join(mat_dir_LOS, p, m + '.mtx')
                    mat_id = 'mf' + str(mat_no)
                    imp_mat.MatrixId = mat_id
                    imp_mat.ImportFile = mat_file
                    imp_mat.MatrixDescription = run_name + ' ' + run_date + ' ' + p + ' ' + m
                    imp_mat.Scenario = scenario
                    imp_mat.run()

                    # export to text file

                    mat_file_out = 'LOSmatrix_'+p + '_' + m + '.csv'
                    matrices_file = os.path.join(processed_dir, mat_file_out)
                    exp_mat(matrices=mat_id,
                        export_file=matrices_file,
                        export_format="PROMPT_DATA_FORMAT",
                        skip_default_values=False,
                        full_matrix_line_format="ONE_ENTRY_PER_LINE",
                        scenario = scenario
                           )   

                    mat_no +=1
            
            # demand matrices
            for m in mat_files_dem:
                
                'Processing demand matrices for {}, {}'.format(p,m)
                
                if p!= 'ON':
                    
                    # import binary matrix
                    mat_file = os.path.join(mat_dir_dem, p+m + '.mtx')
                    mat_id = 'mf' + str(mat_no)
                    imp_mat.MatrixId = mat_id
                    imp_mat.ImportFile = mat_file
                    imp_mat.MatrixDescription = run_name + ' ' + run_date + ' ' + p + ' ' + m[:-6] + ' demand'
                    imp_mat.Scenario = scenario
                    imp_mat.run()

                    # export to text file

                    mat_file_out = 'demand_matrix_'+p + '_' + m[:-6] + '.csv'
                    matrices_file = os.path.join(processed_dir, mat_file_out)
                    exp_mat(matrices=mat_id,
                        export_file=matrices_file,
                        export_format="PROMPT_DATA_FORMAT",
                        skip_default_values=False,
                        full_matrix_line_format="ONE_ENTRY_PER_LINE",
                        scenario = scenario
                           )   

                    mat_no +=1
                    
            
            scen += 2
            
    # get auto metrics

    if process_auto_metrics:

        scen = scen_start

        vkt_scen = pd.DataFrame(index = periods)


        for p in period_order:
            
            p_factor = periods[p]['factor']
            
            'Processing auto metics for {}'.format(p)

            
            emmebank = inro.modeller.Modeller().emmebank
            scenario = emmebank.scenario(scen)

            # set primary scenario
            de.replace_primary_scenario(scenario)

            # create attribute for visual check and filtering
            check_links = ext_att(extra_attribute_type="LINK",
                               extra_attribute_name="@check_ls",
                               extra_attribute_description="link category for Lakeshore",
                               overwrite=True) 
            check_nodes = ext_att(extra_attribute_type="NODE",
                               extra_attribute_name="@lts_area",
                               extra_attribute_description="node category for Lakeshore",
                               overwrite=True)
            check_nodes = ext_att(extra_attribute_type="NODE",
                               extra_attribute_name="@iszone",
                               extra_attribute_description="for shortest path",
                               overwrite=True)
            # screenline attribute
            check_links = ext_att(extra_attribute_type="LINK",
                               extra_attribute_name="@screenline",
                               extra_attribute_description="for screenline analysis",
                               overwrite=True) 

            # get network  
            network = scenario.get_network()
            
            # mark nodes
            for n in network.nodes():
                if int(n.id) in inscope_nodes:
                    n['@lts_area'] = 1
                if n.is_centroid:
                    n['@iszone'] = 1

            # get vkt 
            vkt = 0
            vkt_peel = 0
            vkt_ls_area = 0
            vkt_lakeshore = 0
            for l in network.links():
                l_vkt = l.length * l.auto_volume
                vkt += l_vkt

                if (int(l.i_node.id) >= 40000 and int(l.i_node.id) <= 49999) or (int(l.j_node.id) >= 40000 and int(l.j_node.id) <= 49999): # link in Peel Region
                    vkt_peel += l_vkt
                    
                if int(l.i_node.id) in (inscope_nodes) and int(l.j_node.id) in (inscope_nodes):
                    vkt_ls_area += l_vkt
                    l['@check_ls'] = 1

                if int(l.i_node.id) in (lakeshore_nodes) and int(l.j_node.id) in (lakeshore_nodes):
                    vkt_lakeshore += l_vkt
                    l['@check_ls'] = 2
                    
                # assign screenline value
                if str(l.id) in screenlines.id.to_list():
                    l['@screenline'] = screenlines.loc[screenlines.id == str(l.id), 'screenline']

            vkt_scen.loc[p,'Total'] = vkt /p_factor
            vkt_scen.loc[p,'Peel'] = vkt_peel /p_factor
            vkt_scen.loc[p,'Lakeshore Area'] = vkt_ls_area /p_factor
            vkt_scen.loc[p,'Lakeshore Road'] = vkt_lakeshore /p_factor

            scenario.publish_network(network)

            app = _ap.connect()
            app.refresh_data()

            # get worksheet for speeds and demand

            root_ws_folder = desktop.root_worksheet_folder()
            links_sheet = root_ws_folder.find_item(['Lakeshore','Links_Lakeshore'])

            links = links_sheet.open()
            links.save_as_data_table(name = 'Links_forExport',overwrite = True)
            links.close()
            dts = desktop.project.data_tables()
            links_dt = dts.table('Links_forExport')
            data = links_dt.get_data()
            output_file = os.path.join(processed_dir,'LinksTable_' + p + '.csv')
            data.export_to_csv(output_file, separator = ',')
            
            # get worksheet for links in study area
            
            links_sheet = root_ws_folder.find_item(['Lakeshore','Links_StudyArea'])

            links = links_sheet.open()
            links.save_as_data_table(name = 'Links_forExport',overwrite = True)
            links.close()
            dts = desktop.project.data_tables()
            links_dt = dts.table('Links_forExport')
            data = links_dt.get_data()
            output_file = os.path.join(processed_dir,'LinksStudyArea_' + p + '.csv')
            data.export_to_csv(output_file, separator = ',')
            
            
            # get distance skims
            sp_modes = [scenario.mode('c')]
            sp_out = os.path.join(processed_dir,'ShortestDistance_' + p)
            sp = shortest_path(
                modes = sp_modes,
                roots_attribute="@iszone",
                leafs_attribute="@iszone",
                link_cost_attribute="length",
                num_processors="max-1",
                direction="AUTO",
                costs_file=sp_out,
                return_numpy=False)


            scen += 2 #skip transit scenarios for auto metrics


        vkt_scen.to_csv(os.path.join(processed_dir, 'vkt.csv'), index = True)
        
    # get transit metrics
    if process_transit_metrics: 
        
        scen = scen_start + 1 # skip auto

        pkt_transit_scen = pd.DataFrame(index = periods)

        for p in period_order: 
            
            if p != 'ON': # omit ON
                
#                 p_hours = (periods[p]['end'] - periods[p]['start'] ) / 60 
            
                'Processing transit metics for {}'.format(p)

                scenario_id = scen
                emmebank = inro.modeller.Modeller().emmebank
                scenario = emmebank.scenario(scenario_id)

                # set primary scenario
                de.replace_primary_scenario(scenario)

                # create attribute for visual check and filtering
                check_links = ext_att(extra_attribute_type="TRANSIT_LINE",
                                   extra_attribute_name="@check_ls",
                                   extra_attribute_description="line category for Lakeshore",
                                   overwrite=True)    
                # get network
                network = scenario.get_network()  

                # get pkt transit
                pkt_total = 0
                pkt_peel = 0
                pkt_line23 = 0
                pkt_express = 0
                pkt_line14 = 0
                pkt_line8 = 0
                pkt_line45 = 0
                pkt_line29 = 0
                pkt_line13 = 0
                pkt_line111 = 0
                pkt_line185 = 0
                pkt_line2 = 0
                pkt_hur = 0

                # mark Peel transit
                for li in network.transit_lines():
                    if li.id[0] == 'M' or li.id[0] == 'B':
                        li['@check_ls'] = 1

                for ts in network.transit_segments():

                    if ts.j_node != None:
                        ts_pkt = ts.transit_volume * ts.link.length
                        pkt_total += ts_pkt

                        if ts.line['@check_ls'] == 1:
                            pkt_peel += ts_pkt

                        if 'MLak' in ts.line.id:
                            pkt_express += ts_pkt

                        if 'M023' in ts.line.id:
                            pkt_line23 += ts_pkt

                        if 'M014' in ts.line.id:
                            pkt_line14 += ts_pkt

                        if 'M008' in ts.line.id:
                            pkt_line8 += ts_pkt
                            
                        if 'M045' in ts.line.id:
                            pkt_line45 += ts_pkt
                            
                        if 'M029' in ts.line.id:
                            pkt_line29 += ts_pkt
                            
                        if 'M013' in ts.line.id:
                            pkt_line13 += ts_pkt
                            
                        if 'M111' in ts.line.id:
                            pkt_line111 += ts_pkt
                            
                        if 'M185' in ts.line.id:
                            pkt_line185 += ts_pkt
                            
                        if 'M002' in ts.line.id:
                            pkt_line2 += ts_pkt
                            
                        if 'MHur' in ts.line.id:
                            pkt_hur += ts_pkt



                    pkt_transit_scen.loc[p,'Total'] = pkt_total 
                    pkt_transit_scen.loc[p,'Peel'] = pkt_peel 
                    pkt_transit_scen.loc[p,'Line23'] = pkt_line23 
                    pkt_transit_scen.loc[p,'LakeshoreExpress'] = pkt_express 
                    pkt_transit_scen.loc[p,'Line14'] = pkt_line14 
                    pkt_transit_scen.loc[p,'Line8'] = pkt_line8 
                    pkt_transit_scen.loc[p,'Line45'] = pkt_line45 
                    pkt_transit_scen.loc[p,'Line29'] = pkt_line29 
                    pkt_transit_scen.loc[p,'Line13'] = pkt_line13 
                    pkt_transit_scen.loc[p,'Line111'] = pkt_line111 
                    pkt_transit_scen.loc[p,'Line185'] = pkt_line185 
                    pkt_transit_scen.loc[p,'Line2'] = pkt_line2 
                    pkt_transit_scen.loc[p,'HuLRT'] = pkt_hur 

    #                 scenario.publish_network(network)

                # get worksheet for speeds and demand
                root_ws_folder = desktop.root_worksheet_folder()
                tseg_sheet = root_ws_folder.find_item(['Lakeshore','Segments_LS'])

                tseg = tseg_sheet.open()
                tseg.save_as_data_table(name = 'TransitSegments_forExport',overwrite = True)
                tseg.close()
                dts = desktop.project.data_tables()
                tseg_dt = dts.table('TransitSegments_forExport')
                data = tseg_dt.get_data()
                output_file = os.path.join(processed_dir,'TransitSegmentsTable_' + p + '.csv')
                data.export_to_csv(output_file, separator = ',')

                scen += 2

        pkt_transit_scen.loc['ON',:] = 0

        pkt_transit_scen.to_csv(os.path.join(processed_dir,'pkt_transit.csv'))

#     if process_distance_skims:

#         'Processing distance skims {}'.format('test')

#         scen = scen_start + 1 # skip auto

#         emmebank = inro.modeller.Modeller().emmebank
#         scenario = emmebank.scenario(scen)

#         # set primary scenario
#         de.replace_primary_scenario(scenario)

#         # create attribute for visual check and filtering
#         check_links = ext_att(extra_attribute_type="TRANSIT_LINE",
#                            extra_attribute_name="@check_ls",
#                            extra_attribute_description="line category for Lakeshore",
#                            overwrite=True)    
#         # get network
#         network = scenario.get_network()  

#         # get zones in scenario
#         zones_sc = scenario.zone_numbers

#         # create matrix of ones, zero diagonal
        

                
    
    return

In [8]:
for i in run_properties:
    'Processing {}'.format(run_properties[i]['run_name'])
    process_outputs(run_properties[i])
    
end_t = time()
rt = end_t - start_t
rt_min = int(rt/60)
rt_sec = int(rt%60)
rt_text = '{}:{}'.format(rt_min, rt_sec)
rt_text

'60:9'